# <center> Ансамбли </center>

`Дано:` 
- множество объектов обучающей выборки $X^l = (x_i, y_i)_{i=1}^{l}, x_i\in \mathbb{R}$, разделенных на классы $Y=\{-1,1\}$; 
- множество объектов тестовой выборки, классовая принадлежность которых не известна. 
- $a_t: X\rightarrow Y, t=1,...T$ - обучаемые базовые алгоритмы.

**Алгоритм классификации имеет вид:** 
$$a_t(x)=C(b_t(x))$$
$$a_t: X \overset{b_t}{\rightarrow}R\overset{C}{\rightarrow}Y$$
- $b_t$ - алгоритмический оператор (вероятность принадлежности объекта к классу, расстояние объекта до разделяющей поверхности, степень уверенности классификации и т.п.

*Композиция $T$ алгоритмов $a_t(x)=C(b_t(x)), t=1,...T$* называется суперпозиция алгоритмических операторов $b_t:X\rightarrow R$, корректирующей операции $F:R^T \rightarrow R$ и решающего правила $C:R\rightarrow Y$.

$$a(x)=C(F(b_1(x),...,b_T(X))), x \in X$$

**Примеры агрегирующих функций:** 
- простое голосование
$$F(b_1,...,b_T) = \frac{1}{T}\sum_{t=1}^T b_t$$
- взвешенное голосование 
$$F(b_1,...,b_T)= \sum_{t=1}^T \alpha_t b_t, \quad \sum_{t=1}^T\alpha_t=1, \quad \alpha_t  \geq 0$$
- смесь алгоритмов с функциями компетентности 
$$F(b_1,...,b_T)= \sum_{t=1}^T g_t(x) b_t$$

**Базовые алгоритмы не являются независимыми:** 
- решают одну и ту же задачу;
- настраиваются на один целевой вектор ($y_i$);
- выбираются из одной и той же модели.

**Способы повышения разнообразия базовых алгоритмов:** 
- обучение по различным подвыборкам; 
- обучение по различным наборам признаков; 
- обучение из разных моделей; 
- обучение по с использованием рандомизации: 
    - bagging - формирование подвыборок из обучающей выборки "с возвращением" (длина каждой подвыборки равна длине исходной выборки);
    - pasting - формирование случайной обучающей подвыборки;
    - random subspaces - формирование случайного подмножества признаков;
    - random patches - формирование случайного подмножества и объектов, и признаков;
    - cross-validated committees. 

## Бустинг для задачи классификации

Предположим:
- решающее правило фиксировано $C=sign(b)$;
- $b_t: X \rightarrow \{-1,0,1\}$, $b_t(x)=0$ отказ базового алгоритма от классификации объекта $x$.

Алгоритмическая композиция имеет вид:
$$a(x)=C(F(b_1(x),...,b_T(X)))=sign(\sum_{t=1}^T \alpha_t b_t(x)), x \in X$$

Функционал качества композиции -- количество ошибок на выборке:
$$Q_T = \sum_{i=1}^l [y_i*\sum_{t=1}^T \alpha_t b_t(x)<0]$$

Для упращения задачи минимизации фукнционала $Q_T$ используем следующие эвристики:
- При добавлении в композицию слагаемого $\alpha_tb_t(x)$ оптимизируется только базовый алгоритм $b_t$ и коэффициент при нем $\alpha_t$, а все предыдущие слагаемые $\alpha_1b_1(x),...,\alpha_{t-1}b_{t-1}(x)$ полагаются фиксированными.
- Порогавая функция потерь в функционале $Q_t$ аппроксимируется (заменяется) непрерывно дифференцируемой оценкой сверху. Для adaboost используется экспоненциальная аппроксимация: $[y_ib(x_i)<0]\leq e^{-y_ib(x_i)}$

Тогда функционал $Q_t$ можно оценить сверху:

$$Q_T \leq \widetilde{Q_T} = \sum_{i=1}^l exp(-y_i\sum_{t=1}^T \alpha_t b_t(x_i)) = \sum_{i=1}^l exp(-y_i\sum_{t=1}^{T-1} \alpha_t b_t(x_i)) * exp(-y_i\alpha_T b_T(x_i)) $$

$exp(-y_i\sum_{t=1}^T-1 \alpha_t b_t(x_i))$ -- это вес $w_i$ i-го объекта.

Нормируем все веса и получаем вектор $\widetilde{W}^l = (\tilde{w_1},...,\tilde{w_l})$, где $\tilde{w}^i = \frac{w_i}{\sum_{j=1}^l w_j}$.

Определим два функционала качества:
- суммарный вес ошибочных классификаций $N(b; \widetilde{W}) = \sum_{i=1}^l\tilde{w}[b(x_i)=-y_i]$
- суммарный вес правильных классификаций $P(b; \widetilde{W}) = \sum_{i=1}^l\tilde{w}[b(x_i)=y_i]$

Пусть для любого нормированного вектора весов $\widetilde{W}$ существует базовый алгоритм $b$ классифицирующий выборку хотя бы немного лучше, чем на угад: $P(b; \widetilde{W})>N(b; \widetilde{W})$. Тогда минимум функционала $\widetilde{Q_T}$ достигается при: 
- $b_T = arg \underset{b}{max}\sqrt{P(b; \widetilde{W})} - \sqrt{N(b; \widetilde{W})}$
- $\alpha_T = \frac{1}{2} ln \frac{P(b; \widetilde{W})}{N(b; \widetilde{W})}$

**Алгоритм построения линейной комбинации классификаторов:**

*Вход:* 

- $X^l, Y^l$ - обучающая выборка;
- $T$ - максимальное число базовых алгоритмов;

*Выход:*
- базовые алгоритмы и их веса $\alpha_tb_t, \quad t=1,...,T$

*Алгоритм:*
1. инициализируем вектор весов: $w_i=1/l, \quad i=1,..l;$
2. для всех $t=1,...,T:$
- Обучить базовый алгоритм на наблюдениях $x_i \in X$, взятых с весом $w_i$;
- Определить суммарный вес правильных классификаций $P(b; \widetilde{W})$;
- Оценить важность алгоритма: $\alpha_t = \frac{1}{2} ln \frac{P(b; \widetilde{W})}{N(b; \widetilde{W})}$
- Пересчитать все объектов: $w_i = w_i* exp(-\alpha_t*y_i*b_t(x_i)), \quad i=1,...,l$
- Провести нормировку весов объектов.

## Градиентный бустинг

$$a_T(x)= \sum_{t=1}^T \alpha_t b_t(x), \quad x\in X, \quad b_t:X\rightarrow \mathbb{R}, \quad\alpha_t\in \mathbb{R}_+$$

**Эвристика:** обучаем $\alpha_t, b_t$ при фиксированных прилылущих.

**Критерий качества** с заданной гладкой функцией потерь $\mathcal{L}(b,y)$:
$$Q(\alpha;b;X^l)=\sum_{i=1}^l\mathcal{L}(\sum_{t=1}^{T-1}\alpha_t b_t(x_i)+\alpha b(x_i), y_i)\rightarrow\underset{\alpha, b}{min}$$
- $(a_{T-1,i})_{i=1}^l$ - вектор текущего приближения
- $(a_{T,i})_{i=1}^l$ - вектор следующего приближения

Градиентный метод минимизации $Q(F)\rightarrow min, f\in \mathbb{R}^l$:
- $a_{0,i}:=$ начальное приближение;
- $a_{T,i}:= a_{T-1,i}-\lambda g_i$ начальное приближение, $i=1,...,l$, где $g_i=\mathcal{L}_f'(a_{T-1,i}, y_i)$ - компоненты вектора градиента, $\lambda$- градиентный шаг;

А добавление одного базового алгоритма: 
$a_{T,i}:= a_{T-1,i}-\lambda b(x_i), \quad i=1,...,l$

Тогда будем искать базовый алгоритм, который приближает вектор антиградиента.

**Алгоритм градиентного бустинга:**

*Вход:* 

- $X^l, Y^l$ - обучающая выборка;
- $T$ - максимальное число базовых алгоритмов;
- $\lambda$ - скорость обучения.

*Выход:*
- базовые алгоритмы и их веса $\alpha_tb_t, \quad t=1,...,T$

*Алгоритм:*
1. инициализируем вектор начальный значений: $a_i=1/l, \quad i=1,..l;$
2. для всех $t=1,...,T:$
- Обучить базовый алгоритм, приближающий антиградиент:  
$b_t:=arg\underset{b\in B}{min}\sum_{i=1}^l(b(x_i)+\mathcal{L}'(w_i,y_i))^2$
- найти вес базового алгоритма $\alpha_t$:  
$\alpha_t:=arg\underset{a>0}{min}\sum_{i=1}^l\mathcal{L}(a_i+\lambda b_t(x_i);y_i)$
- обновить значения:  
$a_i:=a_i+\alpha_t b_t(x_i); \quad i=1,...,l$

**Алгоритм градиентного бустинга адаптация под логарифмическую функцию потерь:**
$$Q(w) = \sum_{i=1}^l log_2(1+e^{-y_i*b(x_i) })\rightarrow \underset{w}{min}$$
Антиградиент: 

$\mathcal{L}'(b(x_i),y_i))= b(x_i)*\sigma(-b(x_i)*y_i)$, где $\sigma(z) = \frac{1}{1+e^{-z}}$

*Вход:* 

- $X^l, Y^l$ - обучающая выборка;
- $T$ - максимальное число базовых алгоритмов;
- $\lambda$ - скорость обучения.

*Выход:*
- базовые алгоритмы и их веса $\alpha_tb_t, \quad t=1,...,T$

*Алгоритм:*
1. инициализируем вектор начальный значений: $a_i=1/l, \quad i=1,..l;$
2. для всех $t=1,...,T:$
- Обучить базовый алгоритм на наблюдениях $\mathcal{L}'(b(x_i),y_i))$;  
- Обновить значения: 
$a_i:=a_i+\lambda b_t(x_i); \quad i=1,...,l$

## Градиентный бустинг на деревьях

### XGBoost

В основе алгоритма -- градиентный бустинг. В качестве базовых алгоритмов CART (Classification and Regression Tree) с регуляризацией.

Дерево регрессии и классификации задается суммой: 
$$b(x,w)=\sum_{k in K} w_k B_k(x)$$
где: 
- $B_k(x)$ - бинарный индикатор, который показывает, попало ли наблюдение $x$ в лист $k$ или нет;
- $w_k$ - значение ответа в листе $k$;
- $K$ - множество листьев дерева. 

Критерий качества: 
$$Q(w) = \sum_{i=1}^l L(a(x_i)+b(x_i, w);y_i) + \gamma |K|+\frac{\lambda}{2}\sum_{k\in K}w_k^2 \rightarrow \underset{w}{min}$$
где: 
- $a(x_i) = \sum_{t=1}^{T-1} \alpha_t b_t(x)$ - ранее построенная часть ансамбля.

В результате разложения функции ошибки в ряд Тейлора и вычисления частных производных можно вывести аналитические формулы для вычисления весов $w_k$: 
$$w_k = \frac{\sum_{i=1}^{l} g_iB_k(x_i)}{\lambda+\sum_{i=1}^l h_i B_k(x_i)}$$

Подставив вес в исходную функцию ошибки можно вывести критерий ветвления дерева: 
$$-\frac{1}{2}\sum_{k \in K} \frac{(\sum_{i=1}^{l} g_iB_k(x_i))^2}{\lambda+\sum_{i=1}^l h_i B_k(x_i)} +\gamma|K| \rightarrow min $$

### CatBoost

Статья: https://papers.nips.cc/paper_files/paper/2018/file/14491b756b3a51daac41c24863285549-Paper.pdf

**Мотивации создания:**
1. Необходимо орабатывать категориальные признаки с большим количеством редких значений (полльзователи, регион, город, реклама, рекламодатель, товар, документ, автор и т.д.). 
2. Переобучения в градиентах: $g_i=L'(a_{t-1}(x_i),y_i)$ вычисляется в тех же точках $x_i$, по которым ансамбль $a_{t-1}$ обучался апроксимировать $y_i$.


**Изменения в схеме обучения:**
1. Для получения несмещенных оценок градиента на объекте $x_i$ хранить и дообучать ансамбль на выборках без этого объекта.
2. Для обучения моделей использовать случайные перестановки наблюдений.
3. При кодировании категориальных переменных с большой градацией значений расчитываются статистики объекта $x_i$ по объектам, идущим до $x_i$ в перестановке объектов.
4. На одном уровне в деревьях находятся одинаковые предикаты.
5. Для каждого предиката считается функция качества разбиения, к которой добавляется случайная величина. Чем глубже дерево, тем случайная величина меньше
6. Наращивание перебора в категориальных признаков во время построения дерева.


**Особнности релизации:** 
1. Ненужно делать One-Hot encoding в качестве предобработки.
2. Можно выводить график обучения. 
3. Есть встроенный кросс-валидатор.

### Более сложный пример

In [1]:
!pip install --upgrade numpy==1.26

  Obtaining dependency information for numpy==1.26 from https://files.pythonhosted.org/packages/93/fd/3f826c6d15d3bdcf65b8031e4835c52b7d9c45add25efa2314b53850e1a2/numpy-1.26.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.1 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.1 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.1 kB 330.3 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.1 kB 326.8 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/61.1 kB 327.7 kB/s eta 0:00:01
     -------------------------------------- 61.1/61.1 kB 325.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB 991.0 kB/s eta 0:00:16
   ---------------------------------------- 0.2/15.8 MB 2.1 MB/s eta 0:00:08
    --------------------------------------- 0.3/15.8 MB 2.4 MB/s eta 0:00:07
   - ----------------

ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\PerfLogs\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
pip install matplotlib==3.8

  Obtaining dependency information for matplotlib==3.8 from https://files.pythonhosted.org/packages/40/d9/c1784db9db0d484c8e5deeafbaac0d6ed66e165c6eb4a74fb43a5fa947d9/matplotlib-3.8.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.6 MB 217.9 kB/s eta 0:00:35
   ---------------------------------------- 0.1/7.6 MB 416.7 kB/s eta 0:00:19
    --------------------------------------- 0.2/7.6 MB 701.4 kB/s eta 0:00:11
   - -------------------------------------- 0.3/7.6 MB 1.1 MB/s eta 0:00:07
   -- ------------------------------------- 0.6/7.6 MB 1.8 MB/s eta 0:00:05
   ---- ----------------------------------- 0.9/7.6 MB 2.4 MB/s eta 0:00:03
   ------ --------------------------------- 1.2/7.6 MB 3.0 MB/s eta 0:00:03
   ------- -------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 3.3.2 requires matplotlib<3.8.0, but you have matplotlib 3.8.0 which is incompatible.


In [3]:
from IPython.display import display
import ipywidgets as widgets
display(widgets.HTML('<a>1</a>'))

HTML(value='<a>1</a>')

In [4]:
pip install --upgrade ipywidgets

  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/22/2d/9c0b76f2f9cc0ebede1b9371b6f317243028ed60b90705863d493bae622e/ipywidgets-8.1.5-py3-none-any.whl.metadata
  Obtaining dependency information for comm>=0.1.3 from https://files.pythonhosted.org/packages/e6/75/49e5bfe642f71f272236b5b2d2691cf915a7283cc0ceda56357b61daa538/comm-0.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.12 from https://files.pythonhosted.org/packages/21/02/88b65cc394961a60c43c70517066b6b679738caf78506a5da7b88ffcb643/widgetsnbextension-4.0.13-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.12 from https://files.pythonhosted.org/packages/a9/93/858e87edc634d628e5d752ba944c2833133a28fa87bb093e6832ced36a3e/jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/139.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/139.8 kB ? eta -

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 3.3.2 requires matplotlib<3.8.0, but you have matplotlib 3.8.0 which is incompatible.


In [5]:
from sklearn.datasets import make_circles, make_classification, make_moons
import numpy as np
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.pipeline import make_pipeline
import plotly.express as px
import plotly.graph_objects as go
#from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, log_loss
from catboost import CatBoostClassifier
from ucimlrepo import fetch_ucirepo 
from sklearn.preprocessing import LabelEncoder
#import matplotlib.pyplot as plt

In [6]:
adult = fetch_ucirepo(id=2) 
X = adult.data.features 
y = adult.data.targets 
le = LabelEncoder()

In [7]:
y = y.replace({'<=50K.':'<=50K', '>50K.':'>50K'})

In [8]:
X.fillna(value = 'None', inplace = True)

C:\Users\Артур\AppData\Local\Temp\ipykernel_6864\2537910290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(value = 'None', inplace = True)


In [9]:
cat_features = X.columns[X.dtypes == 'object'].values
cat_features

array(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'], dtype=object)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [11]:
bst = CatBoostClassifier(
    iterations = 100, 
    depth = 3, 
    learning_rate = 1, 
    loss_function = 'Logloss', 
    verbose = False, 
    cat_features = cat_features,
    custom_loss=['AUC', 'Accuracy'])

In [12]:
bst.fit(X_train, y_train, 
        eval_set = (X_test, y_test), 
        plot = True, 
        plot_file = '1.html')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [13]:
from sklearn.metrics import classification_report, confusion_matrix

In [14]:
predict = bst.predict(X_test)

In [15]:
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

       <=50K       0.90      0.94      0.92      7401
        >50K       0.78      0.66      0.71      2368

    accuracy                           0.87      9769
   macro avg       0.84      0.80      0.82      9769
weighted avg       0.87      0.87      0.87      9769



In [16]:
bst.classes_

array(['<=50K', '>50K'], dtype=object)

In [17]:
confusion_matrix(y_test, predict)

array([[6973,  428],
       [ 813, 1555]], dtype=int64)

### Кросс-валидация

In [18]:
from catboost import cv, Pool

In [19]:
params = {'loss_function':'Logloss', 
          'iterations': 150,
          'custom_loss': 'AUC', 
          'learning_rate':0.5}

In [20]:
cv_data = cv(
    params = params, 
    pool = Pool(X_train, y_train, cat_features = cat_features),
    fold_count = 5, 
    shuffle = True, 
    plot = True,
    verbose = False
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 0.2764726883
bestIteration = 51

Training on fold [1/5]

bestTest = 0.2875569674
bestIteration = 55

Training on fold [2/5]

bestTest = 0.273469701
bestIteration = 71

Training on fold [3/5]

bestTest = 0.2811539887
bestIteration = 62

Training on fold [4/5]

bestTest = 0.2802325538
bestIteration = 71



In [21]:
cv_data

,iterations,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std,test-AUC-mean,test-AUC-std
0,0,0.389039,0.009155,0.388222,0.005158,0.882015,0.008844
1,1,0.340393,0.006441,0.339323,0.001407,0.895998,0.005868
2,2,0.319941,0.006411,0.318619,0.002426,0.904984,0.005248
3,3,0.311861,0.007315,0.310311,0.002155,0.908830,0.005027
4,4,0.307096,0.006264,0.305258,0.001713,0.910898,0.004212
...,...,...,...,...,...,...,...
145,145,0.285881,0.005123,0.225783,0.000978,0.924055,0.002904
146,146,0.285830,0.005254,0.225464,0.001087,0.924126,0.002973
147,147,0.285940,0.005329,0.225261,0.001048,0.924074,0.003012
148,148,0.286219,0.005207,0.225026,0.001081,0.923954,0.002977


In [22]:
bv = cv_data['test-Logloss-mean'].min()
bi = cv_data['test-Logloss-mean'].argmin()
print(f'Лучший log-loss:{bv:.3f}+/-{cv_data["test-Logloss-std"][bi]:.3f} получен на итерации {bi}')

Лучший log-loss:0.280+/-0.006 получен на итерации 71


### Ранняя остановка

In [23]:
model_2  = CatBoostClassifier(
    iterations = 100, 
    depth = 5, 
    learning_rate = 1, 
    loss_function = 'Logloss', 
    verbose = False, 
    cat_features = cat_features,
    custom_loss=['AUC', 'Accuracy'], 
    train_dir = 'depth_5',
    early_stopping_rounds = 20)

In [24]:
model_2.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False, plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [25]:
model_3  = CatBoostClassifier(
    iterations = 100, 
    depth = 5, 
    learning_rate = 1, 
    loss_function = 'Logloss', 
    eval_metric = 'AUC',
    verbose = False, 
    cat_features = cat_features,
    custom_loss=['AUC', 'Accuracy'], 
    train_dir = 'depth_5',
    early_stopping_rounds = 20)

model_3.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False, plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### Выбор порога предсказания

In [26]:
from catboost.utils import get_roc_curve, get_fnr_curve, get_fnr_curve, select_threshold
from sklearn.metrics import auc

In [27]:
eval_pool = Pool(X_test, y_test, cat_features= cat_features)
fpr, tpr, treshold = get_roc_curve(model_3, eval_pool)
roc_auc = auc(fpr, tpr)
roc_auc

0.9301354170090236

### Анализ предсказаний

In [28]:
train_pool = Pool(X_train, y_train, cat_features= cat_features)

In [29]:
model_3.calc_feature_statistics(train_pool,
                                feature = 2,
                                plot=True,
                        thread_count=-1,
                        plot_file='2.html')

{'borders': array([ 25943.5,  31843. ,  32985. ,  40242.5,  48883.5,  55496. ,
         67314. ,  97681.5, 136269.5, 152838. , 182273.5, 192354.5,
        193042.5, 202871.5, 213308.5, 216606. , 226745.5, 242499.5,
        274500.5, 314398.5, 379523.5, 472030. ], dtype=float32),
 'binarized_feature': array([ 6, 19, 16, ..., 18, 11, 20]),
 'mean_target': array([0.2631579 , 0.22950819, 0.20915033, 0.22270742, 0.22131148,
        0.19836065, 0.17169069, 0.22922637, 0.25487232, 0.25665167,
        0.2498635 , 0.2712975 , 0.3026316 , 0.22502522, 0.25519288,
        0.22319475, 0.22831424, 0.2254025 , 0.21048321, 0.20995198,
        0.25713184, 0.23233795, 0.19016394], dtype=float32),
 'mean_weighted_target': array([], dtype=float32),
 'mean_prediction': array([0.26139474, 0.22716597, 0.2377719 , 0.21572691, 0.20337133,
        0.21651602, 0.17073047, 0.23513053, 0.2555849 , 0.25515315,
        0.25081408, 0.2640809 , 0.28997222, 0.2335273 , 0.2477091 ,
        0.2165487 , 0.23323809, 0.2264

### Оценка важности признаков

#### Возможности catboost

In [30]:
from catboost import Pool

In [31]:
pool_train = Pool(X_train, y_train, cat_features = cat_features)

In [32]:
bst.feature_importances_

array([11.97776322,  2.32865862,  1.23660486,  1.63451328, 10.33415421,
        7.18243555,  7.23259136, 23.09712125,  0.42799311,  1.74886706,
       17.30133412,  7.38469161,  7.27171332,  0.84155843])

In [33]:
featureImportance = bst.get_feature_importance(pool_train, type = 'FeatureImportance', prettified = True)
featureImportance.head()

,Feature Id,Importances
0,relationship,23.019224
1,capital-gain,17.378155
2,age,12.030345
3,education-num,10.390562
4,capital-loss,7.428953


In [34]:
px.bar(x = featureImportance['Importances'], y = featureImportance['Feature Id'])

In [35]:
shap = bst.get_feature_importance(pool_train, type = 'ShapValues', prettified=True)
shap

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.633671,0.056288,-0.344578,-0.273096,1.217016,0.912149,0.497955,1.503201,0.027727,-0.111298,-0.236244,-0.058681,-0.707070,0.028860,-2.456254
1,-1.503422,-0.024647,-0.406779,0.077778,-0.399893,-0.072412,0.035387,-1.249724,0.035029,0.214685,-0.143022,-0.046239,0.892332,-0.004330,-2.456254
2,-0.825149,-0.035353,0.100862,0.129772,-0.103034,-0.288427,-0.152388,-1.164221,0.035029,-0.216072,-0.143022,-0.036691,-0.275227,0.028345,-2.456254
3,1.007776,0.015214,-0.052576,0.058207,-0.132994,-0.361790,-0.705742,-1.257321,-0.300930,0.306088,-0.140954,-0.044048,-0.113338,0.029181,-2.456254
4,0.699763,0.031791,0.111761,0.030611,-0.054109,0.394355,0.869059,-1.183948,0.035029,-0.319476,-0.170729,-0.056301,0.995087,0.029181,-2.456254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39068,0.714437,-0.263286,-0.003645,0.076798,-1.196704,0.584031,-0.496908,0.553416,0.027727,0.114244,-0.215800,-0.059976,0.601880,0.021747,-2.456254
39069,0.282441,0.052831,-0.035313,0.138663,-1.685094,0.882825,-0.383419,0.729867,0.028532,-0.240120,-0.988117,-0.061088,0.518847,-0.478755,-2.456254
39070,0.497489,-0.031910,-0.093661,0.049665,-1.608203,0.876220,-0.477331,0.631132,-0.214238,0.081900,-0.228925,-0.047611,0.036066,-0.770181,-2.456254
39071,0.788257,-0.289470,0.056317,0.083682,-0.428034,0.743747,-0.927124,0.750002,0.028532,0.114244,-0.215800,-0.047179,0.010386,0.022026,-2.456254


In [36]:
interaction = bst.get_feature_importance(pool_train, type = 'Interaction', prettified=True)
interaction.head()

,First Feature Index,Second Feature Index,Interaction
0,0,11,5.972428
1,7,11,4.189715
2,5,7,3.582008
3,0,10,2.984106
4,1,6,2.928696


In [37]:
px.scatter(x = interaction['First Feature Index'], y = interaction['Second Feature Index'], size = interaction['Interaction'])

In [38]:
crosstab = pd.crosstab(index = interaction['First Feature Index'], 
                       columns = interaction['Second Feature Index'], 
                       values = interaction['Interaction'], aggfunc = 'sum')
crosstab

Second Feature Index,1,2,3,4,5,6,7,8,10,11,12,13
First Feature Index,,,,,,,,,,,,
0,1.707573,2.360308,2.407137,0.486064,0.657876,2.187368,0.706961,NaN,2.984106,5.972428,2.781203,2.135703
1,NaN,0.541880,0.853353,0.072043,NaN,2.928696,0.763740,NaN,NaN,0.528852,1.015984,1.056318
2,NaN,NaN,2.006755,1.332225,1.050464,0.152329,0.859786,0.666885,NaN,2.206261,0.858793,0.357732
3,NaN,NaN,NaN,1.375005,2.355188,1.621781,0.341510,0.659599,1.031039,1.628875,0.284760,0.541568
4,NaN,NaN,NaN,NaN,2.913074,0.639521,0.897166,0.568225,0.849591,NaN,0.427421,NaN
5,NaN,NaN,NaN,NaN,NaN,0.379688,3.582008,1.389562,2.838435,1.515368,2.375518,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,0.958343,0.366927,0.093659,1.404892,2.444758,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.809010,1.332789,4.189715,2.400218,0.560106
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.835305,1.656675,NaN,0.514802


In [39]:
dict_names = {i: j for i,j in enumerate(bst.feature_names_)}

In [40]:
px.imshow(crosstab, x = crosstab.columns.map(dict_names),y =crosstab.index.map(dict_names))

In [41]:
from sklearn.inspection import permutation_importance
r = permutation_importance(bst, X_test, y_test,
                           n_repeats=30,
                           random_state=0)

In [42]:
importances = [[y, x] for x,y in zip(r['importances_mean'], X_test.columns)]

In [43]:
importances.sort(key = lambda x: x[1])

In [44]:
importances= np.transpose(importances)

In [45]:
px.histogram(y = importances[0], x = importances[1], orientation = 'h')

#### Библиотека shap

In [ ]:
!pip install --upgrade numpy

In [49]:
!pip install shap --user

In [50]:
import shap

ImportError: Numba needs NumPy 1.24 or less

In [ ]:
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(bst)

In [ ]:
shap_values = explainer(X_test)

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
shap.summary_plot(shap_values, X_test)

In [ ]:
shap_values[0]

In [ ]:
i=3
print(bst.predict(X_test)[i],y_test.values[i])
shap.waterfall_plot(shap_values[i])


In [ ]:
shap.force_plot(shap_values[0])

In [ ]:
not_equal = [i for i,j in enumerate(zip(bst.predict(X_test),y_test.values)) if j[0]!=j[1]]

In [ ]:
len(not_equal)

In [ ]:
def update_plot(instanse_index=2):
    item = not_equal[instanse_index]
    print(y_test.iloc[item])
    shap.force_plot(shap_values[item], matplotlib = True)

In [ ]:
from ipywidgets import interact

In [ ]:
interact(update_plot, instanse_index =(0, len(not_equal),1));

In [ ]:
shap.unique(explainer.expected_value, explainer.shap_values(X_test)[not_equal], X_test.iloc[not_equal,:])

In [ ]:
X_test.head()

In [ ]:
feature_shap_values = shap_values[:,9].values
feature_values = X_test['sex'].values

In [ ]:
res = pd.DataFrame([feature_values, feature_shap_values]).T.groupby(0).agg(lambda x: list(x)).to_dict()

In [ ]:
shap_values_feature = pd.DataFrame({i: pd.Series(j) for i,j in res[1].items()})

In [ ]:
px.box(shap_values_feature)

In [ ]:
shap.dependence_plot("sex", shap_values.values, X_test, interaction_index="relationship")

In [ ]:
shap_values_errors = explainer.shap_values(X_test.iloc[not_equal,:])

In [ ]:
y_test.iloc[not_equal,:].value_counts()

In [ ]:
np.unique(bst.predict(X_test)[not_equal], return_counts=True)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values_errors, X_test.iloc[not_equal,:])

## Задание

Провести анализ лучшей модели категориального бустинга, построенной для прогнозирования вероятности выполнения погрузки.